In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Шаг 1: Загрузка изображений
image_paths = ["./image/0.jpg", "./image/2.jpg", "./image/3.jpeg", "./image/4.jpg", "./image/5.jpg", "./image/6.jpg", "./image/7.jpg", "./image/8.jpg", "./image/9.jpg", "./image/10.jpg", "./image/images.jpg", "./image/images1.jpg", "./image/images2.jpg", "./image/disayn-gostinoy-v-sovremennom-stile-3686-125.jpg"]  # Убедитесь, что пути к изображениям корректные
images = []

# Проверка загрузки изображений
for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        print(f"Ошибка: изображение не загружено: {path}")
    else:
        images.append(img)

# Шаг 2: Поиск гипотез с помощью Selective Search
def selective_search(img):
    if img is None or not isinstance(img, np.ndarray):
        raise ValueError("Некорректное изображение передано в selective_search.")
    
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    return rects

# Шаг 3: Функция для извлечения регионов
def extract_regions(img, rects, size=(128, 128)):
    regions = []
    for (x, y, w, h) in rects:
        roi = img[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, size)
        regions.append(roi_resized)
    return regions

# Шаг 4: Извлечение признаков HOG из изображений
def extract_hog_features(image):
    gray_image = rgb2gray(image)  # Преобразуем изображение в оттенки серого
    features, hog_image = hog(gray_image, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), visualize=True, feature_vector=True)
    return features

# Шаг 5: Генерация гипотез и извлечение признаков
hypothesis_count = []
features = []
labels = []  # Метки: 1 - кот, 0 - не кот

for img in images:
    if img is not None:
        rects = selective_search(img)
        regions = extract_regions(img, rects)
        hypothesis_count.append(len(rects))  # Сколько гипотез предложено

        for region in regions:
            hog_features = extract_hog_features(region)
            features.append(hog_features)

            # Лейблы: коты обозначаются меткой 1, остальные регионы - меткой 0 (нужно вручную или из набора данных)
            labels.append(1)  # Здесь можно вручную или автоматически метки присваивать

# Шаг 6: Обучение бинарного классификатора (например, SVM)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

svm = SVC(kernel='linear')  # Используем линейную SVM
svm.fit(X_train, y_train)

# Шаг 7: Предсказание и оценка точности
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Точность классификатора: {accuracy:.2f}")

# Шаг 8: Вывод результатов
for i, img in enumerate(images):
    print(f"Изображение {i+1}: предложено {hypothesis_count[i]} гипотез.")


ValueError: The number of classes has to be greater than one; got 1 class